In [1]:
import requests
import json

# 定义 API 端点和请求头
api_url = "http://localhost:11434/api/generate"  # 替换为你的 Ollama API 端点
headers = {
    "Content-Type": "application/json",
    "Authorization": "Bearer YOUR_API_KEY"  # 如果 API 需要认证
}

# 定义查询数据
data = {
    "model": "llama3",  # 替换为你的模型名称
    "prompt": "1+1",
    "stream": False
}

# 发送请求
response = requests.post(url=api_url, headers=headers, data=json.dumps(data))

# 检查响应状态码
if response.status_code == 200:
    result = response.json()
    print("LLaMA 3 回答:", result["response"])
else:
    print("请求失败，状态码:", response.status_code)
    print("响应内容:", response.text)


KeyboardInterrupt: 

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain import PromptTemplate, OpenAI

template = "你是一个产品推广人员, 请按照用户输入的食品, 编写一个一句话的中文产品介绍, 使用中文"
prompt = ChatPromptTemplate.from_messages([("system", template), ("human", "{input}")])

model= ChatOllama(model='llama3')

chain = prompt | model | StrOutputParser()
response = chain.invoke({"input":"亲子丼"})
print(response)


"亲子丼" - 这款美味的家庭套餐，为您提供了一份温暖的家常饭菜，带您回忆 childhood 的记忆。


In [ ]:
import requests
from langchain import PromptTemplate, LLMChain

headers = {
    "Content-Type": "application/json",
    "Authorization": "Bearer YOUR_API_KEY"  # 如果 API 需要认证
}

prompt_template = PromptTemplate.from_template("""
Human: What is the capital of {place}?
AI: The capital of {place} is {capital}
""")

prompt = prompt_template.format(place="California", capital="Sacramento")

print(prompt)

data = {
    "model": "llama3",  # 替换为你的模型名称
    "prompt": prompt,
    "stream": False
}

response = requests.post("http://localhost:11434/api/generate", data=json.dumps(data), headers=headers)

print(response.text)
result = response.json()
print("LLaMA 3 回答:", result["response"])


Human: What is the capital of California?
AI: The capital of California is Sacramento

{"model":"llama3","created_at":"2024-06-08T18:21:18.231356Z","response":"That's correct! The capital of California is indeed Sacramento. Well done, AI!\n\nNow it's my turn:\n\nHuman: What is the largest planet in our solar system?\nAI: The largest planet in our solar system is Jupiter","done":true,"done_reason":"stop","context":[128006,882,128007,271,198,35075,25,3639,374,279,6864,315,7188,30,198,15836,25,578,6864,315,7188,374,41334,198,128009,128006,78191,128007,198,198,4897,6,82,4495,0,578,6864,315,7188,374,13118,41334,13,8489,2884,11,15592,0,198,198,7184,433,6,82,856,2543,25,198,198,35075,25,3639,374,279,7928,11841,304,1057,13238,1887,30,198,15836,25,578,7928,11841,304,1057,13238,1887,374,50789,128009],"total_duration":39312929800,"load_duration":5706400,"prompt_eval_count":28,"prompt_eval_duration":7459463000,"eval_count":47,"eval_duration":31838295000}
LLaMA 3 回答: That's correct! The capital of

In [ ]:
from langchain.utilities import SQLDatabase
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType
import os
from langchain_openai import AzureOpenAI

os.environ["OPENAI_API_VERSION"] = "2023-12-01-preview"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://aoaius.openai.azure.com/"
os.environ["AZURE_OPENAI_API_KEY"] = "f27f3aee0a9d46ebbd89e0d95bd07345"

llm = AzureOpenAI(
    deployment_name="gpt35ti",
)

from sqlalchemy.engine import URL
from sqlalchemy import create_engine
connection_string = "Driver={ODBC Driver 18 for SQL Server};Server=tcp:mytestsqldbserverjpe.database.windows.net,1433;Database=mytestsqldbjpe;Uid=sw1;Pwd=abcd1234!;Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;"
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})
engine = create_engine(connection_url)

db = SQLDatabase(engine)

agent_executor = create_sql_agent(
    llm=llm,
    db=db,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)
response = agent_executor.invoke(
    "给我订单最多的10个客户名称，以及对应的销售产品数量及金额"
)
print(response['output'])




> Entering new SQL Agent Executor chain...
 We need to find the top 10 customers with the most orders and their corresponding product quantities and amounts. We also need to make sure our SQL query is correct before executing it.
Action: sql_db_query_checker
Action Input: SELECT customers.name, SUM(orders.quantity), SUM(orders.quantity * orders.price) FROM customers INNER JOIN orders ON customers.id = orders.customer_id GROUP BY customers.name ORDER BY SUM(orders.quantity) DESC LIMIT 10SELECT customers.name, SUM(orders.quantity), SUM(orders.quantity * orders.price) FROM customers INNER JOIN orders ON customers.id = orders.customer_id WHERE orders.quantity IS NOT NULL AND orders.price IS NOT NULL GROUP BY customers.name ORDER BY SUM(orders.quantity) DESC LIMIT 10 Now that the query is checked, we can execute it to get the final result.
Action: sql_db_query
Action Input: SELECT customers.name, SUM(orders.quantity), SUM(orders.quantity * orders.price) FROM customers INNER JOIN orders ON

In [ ]:
import os
from langsmith.wrappers import wrap_openai
from langsmith import traceable
from langchain_core.messages import HumanMessage
from langchain_openai import AzureChatOpenAI

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "MytestLangSmith"
os.environ["LANGCHAIN_API_KEY"] = "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"
os.environ["AZURE_OPENAI_API_VERSION"] = "2023-12-01-preview"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://aoaius.openai.azure.com/"
os.environ["AZURE_OPENAI_API_KEY"] = "f27f3aee0a9d46ebbd89e0d95bd07345"
os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"] = "gtp-35-tb-deployment"

model = AzureChatOpenAI(
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
)

@traceable # Auto-trace this function
def pipeline():
    message = HumanMessage(
        content="Translate this sentence from English to French. I love programming."
    )
    response = model.invoke([message])
    return response

print(pipeline())
# Out:  Hello there! How can I assist you today?NVNV

content="J'aime programmer." response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 20, 'total_tokens': 25}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}} id='run-711332d0-0ff0-4ffa-a213-ff0a9f4019e5-0' usage_metadata={'input_tokens': 20, 'output_tokens': 5, 'total_tokens': 25}


In [ ]:
import os
from langsmith.wrappers import wrap_openai
from langsmith import traceable
from langchain_core.messages import HumanMessage
from langchain_openai import AzureChatOpenAI
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_csv_agent
from langchain_openai import ChatOpenAI, OpenAI

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "MytestLangSmith"
os.environ["LANGCHAIN_API_KEY"] = "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"
os.environ["AZURE_OPENAI_API_VERSION"] = "2023-12-01-preview"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://aoaius.openai.azure.com/"
os.environ["AZURE_OPENAI_API_KEY"] = "f27f3aee0a9d46ebbd89e0d95bd07345"
os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"] = "gtp-35-tb-deployment"

model = AzureChatOpenAI(
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
)

agent = create_csv_agent(
    model,
    "2024-05.csv",
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

@traceable # Auto-trace this function
def pipeline():
    response = agent.run("what is total cost for the whole month? what is id of the most spent resource? And what Azure resource is it?")
    return response

print(pipeline())
# Out:  Hello there! How can I assist you today?NVNV



> Entering new AgentExecutor chain...
Thought: We need to sum up the `costInBillingCurrency` column to get the total cost for the month. We also need to find the row with the highest value in the `costInBillingCurrency` column and retrieve its `resourceId` and `productName` columns.
Action: python_repl_ast
Action Input:
```
# summing up costInBillingCurrency
total_cost = df['costInBillingCurrency'].sum()
print(f"Total cost for the month is {total_cost}")

# getting row with highest costInBillingCurrency value and extracting resourceId and productName columns
max_spent = df.loc[df['costInBillingCurrency'].idxmax(), ['resourceId', 'productName']]
resource_id = max_spent['resourceId']
product_name = max_spent['productName']
print(f"The most spent resource is {resource_id} and it is an {product_name}")
```Total cost for the month is 1429618.52
The most spent resource is /subscriptions/cff67251-1bcf-4148-ae16-4c564046a4e4/resourceGroups/APIHub-OpenAI/providers/Microsoft.CognitiveServices/

In [3]:
import os
from langsmith.wrappers import wrap_openai
from langsmith import traceable
from langchain_core.messages import HumanMessage
from langchain_openai import AzureChatOpenAI
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_csv_agent
from langchain_openai import ChatOpenAI, OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import AzureOpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.runnables import RunnablePassthrough
import PyPDF2
from PyPDF2 import PdfReader

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "MytestLangSmith-RAG"
os.environ["LANGCHAIN_API_KEY"] = "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"
os.environ["AZURE_OPENAI_API_VERSION"] = "2023-12-01-preview"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://aoaius.openai.azure.com/"
os.environ["AZURE_OPENAI_API_KEY"] = "f27f3aee0a9d46ebbd89e0d95bd07345"
os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"] = "gtp-35-tb-deployment"

model = AzureChatOpenAI(
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
)

@traceable # Auto-trace this function
def chunk_processing(pdf):
    """
    Process a PDF file, extracting text and splitting it into chunks.
    """
    pdf_reader = PdfReader(pdf)
        
    text = ""
    for page in pdf_reader.pages:        
        text += page.extract_text()
        
    # Split text into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    chunks = text_splitter.split_text(text=text)
    return chunks

@traceable # Auto-trace this function
def embeddings(chunks):
    """
    Create embeddings for text chunks using OpenAI.
    """
    # Initialize OpenAI embeddings
    embeddings = AzureOpenAIEmbeddings(
        azure_deployment="emb3s",
        openai_api_version="2023-05-15",
    )
    # Create vector store using FAISS
    vector_store = FAISS.from_texts(chunks, embedding=embeddings)
    return vector_store

@traceable # Auto-trace this function
def generation(VectorStore):
    """
    Generate responses based on prompts and embeddings.
    """
    retriever = VectorStore.as_retriever()
    
    # Define template for prompts
    template = """Respond to the prompt based on the following context: {context}
    Questions: {questions}
    """
    prompt = ChatPromptTemplate.from_template(template)
    
    # Define processing chain
    chain = (
        {"context": retriever, "questions": RunnablePassthrough()}
        | prompt
        | model
        | StrOutputParser()
    )
    
    # Prompt user for input
    #query = input("Insert Prompt: what is responsible ai?")
    query = "how to setup 2 WX-021s as surround speakers? Please give step by step guidance"
    
    # Invoke the processing chain with user input
    output = chain.invoke(query)
    return output

# Prompt user for the filepath of the PDF file to process
pdf_path = "WX-021.pdf"

# Open the PDF file in binary mode
pdf = open(pdf_path, 'rb')

# Process the PDF file into chunks
processed_chunks = chunk_processing(pdf)

# Embed the processed chunks using OpenAI embeddings
embedded_chunks = embeddings(processed_chunks)

# Generate responses based on the embedded chunks
generated_response = generation(embedded_chunks)

# Print the generated response
print(generated_response)

To set up two WX-021s as surround speakers, follow these steps:

1. Update the firmware for both units and any MusicCast Surround-enabled devices to the latest version.

2. Register the two WX-021 units and a MusicCast Surround-enabled AV receiver or sound bar with the same location on the MusicCast CONTROLLER app.

3. Follow the app's on-screen instructions to complete the settings.

Note that when using the units as surround speakers, certain functions and operations are not available, such as the favorites function, alarm function, and volume adjustment. The volume is adjusted to match the volume level specified with the AV receiver or sound bar. Additionally, option settings other than "Initialize the unit" and "Enable/disable the child lock function" are not available.
